In [1]:
import torch
import datasets
import transformers
import numpy as np
from seqeval.metrics import f1_score
import pandas as pd
import os
from torch.utils.data import DataLoader

In [2]:
from train_datasets import BPEDropoutTrainDataset

In [3]:
if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


In [4]:
cache_dir = "./cache"

# Load Data

In [64]:
dataset_path = "masakhaner"
language = "luo"
# dataset_path = "conll2003"
# language = None

# subword regularization params:
bpe_dropout_p = 0.1
# model_path = "xlm-roberta-base"
model_path = "Davlan/afro-xlmr-mini"

In [65]:
train_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, bpe_dropout_p=bpe_dropout_p, cache_dir=cache_dir, train=True)
test_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, cache_dir=cache_dir, train=False)


Found cached dataset masakhaner (/atlas2/u/xiluo/temp/cache/masakhaner/luo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset masakhaner (/atlas2/u/xiluo/temp/cache/masakhaner/luo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [66]:
tags = train_dataset.dset['train'].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None)

In [67]:
tokenizer = train_dataset.tokenizer

In [68]:
pd.DataFrame(
    [tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']), train_dataset[0]['labels']],
    index=["tokens", "ner_tags"])

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
tokens,<s>,▁Ji,i,▁2,▁mo,ko,▁jo,wi,to,▁ng,...,▁ew,ii,▁od,hi,ambo,▁maka,wu,on,oni,</s>
ner_tags,-100,0,-100,0,0,-100,0,-100,-100,0,...,-100,7,-100,-100,8,-100,-100,-100,-100,-100


# Training

In [69]:
# Make debugging easier
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [70]:
training_args = transformers.TrainingArguments(
    output_dir = "./checkpoints/xlm-roberta-ner-yor-swreg",
    log_level = "error",
    num_train_epochs = 50,
    per_device_train_batch_size = 12,
    per_device_eval_batch_size = 12,
    evaluation_strategy = "epoch",
    fp16 = True,
    logging_steps = len(train_dataset),
    push_to_hub = False,
)

In [71]:
def metrics_func(eval_arg):
    preds = np.argmax(eval_arg.predictions, axis=2)
    batch_size, seq_len = preds.shape
    y_true, y_pred = [], []
    for b in range(batch_size):
        true_label, pred_label = [], []
        for s in range(seq_len):
            if eval_arg.label_ids[b, s] != -100:  # -100 must be ignored
                true_label.append(index2tag[eval_arg.label_ids[b][s]])
                pred_label.append(index2tag[preds[b][s]])
        y_true.append(true_label)
        y_pred.append(pred_label)
    return {"f1": f1_score(y_true, y_pred)}

In [72]:
data_collator = transformers.DataCollatorForTokenClassification(
    tokenizer,
    return_tensors="pt")

In [73]:
xlmr_config = transformers.AutoConfig.from_pretrained(
    model_path,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)

In [74]:
model = (transformers.RobertaForTokenClassification
         .from_pretrained(model_path, config=xlmr_config, cache_dir=cache_dir)
         .to(device))

In [75]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [76]:
trainer.train()

/deep/u/xiluo/anaconda3/envs/swreg/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'eval_loss': 1.3795737028121948, 'eval_f1': 0.0, 'eval_runtime': 1.7725, 'eval_samples_per_second': 104.937, 'eval_steps_per_second': 9.027, 'epoch': 1.0}
{'eval_loss': 1.2250159978866577, 'eval_f1': 0.0, 'eval_runtime': 1.8005, 'eval_samples_per_second': 103.302, 'eval_steps_per_second': 8.886, 'epoch': 2.0}
{'eval_loss': 1.0837069749832153, 'eval_f1': 0.0, 'eval_runtime': 1.8087, 'eval_samples_per_second': 102.834, 'eval_steps_per_second': 8.846, 'epoch': 3.0}
{'eval_loss': 0.962010383605957, 'eval_f1': 0.0, 'eval_runtime': 1.8049, 'eval_samples_per_second': 103.056, 'eval_steps_per_second': 8.865, 'epoch': 4.0}
{'eval_loss': 0.8082224726676941, 'eval_f1': 0.0, 'eval_runtime': 1.802, 'eval_samples_per_second': 103.218, 'eval_steps_per_second': 8.879, 'epoch': 5.0}
{'eval_loss': 0.7160309553146362, 'eval_f1': 0.18813314037626627, 'eval_runtime': 1.7885, 'eval_samples_per_second': 103.998, 'eval_steps_per_second': 8.946, 'epoch': 6.0}
{'eval_loss': 0.6617061495780945, 'eval_f1': 0.171

TrainOutput(global_step=2700, training_loss=0.30914306605303726, metrics={'train_runtime': 1268.8854, 'train_samples_per_second': 25.377, 'train_steps_per_second': 2.128, 'train_loss': 0.30914306605303726, 'epoch': 50.0})